
# Model Training for Stock Prediction using GRU

This notebook is based on the `model_training.py` script and demonstrates the process of setting up and training a GRU (Gated Recurrent Unit) neural network model to predict the evolution of S&P 500 stock prices. It includes details on importing libraries, loading configurations, defining the network architecture, and placeholders for data preparation, model training, and evaluation.


In [ ]:

import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import yaml



## 1. Importing Libraries and Configuration

The first step involves importing necessary libraries and loading the configuration settings from a YAML file. To install the yaml library, execute the command `pip install pyyaml` in the console of your IDE.


In [ ]:

# Load the configuration file
with open('config/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

learning_rate = config['learning_rate']
batch_size = config['batch_size']



## 2. Defining the GRU Network

The `GRUNet` class is defined to create a GRU neural network. This network includes a GRU layer and a fully connected layer. The network is initialized with input dimensions, hidden dimensions, output dimensions, and the number of layers. The forward method defines how the network processes input data.


In [ ]:

class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # GRU Layer
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        
        # Fully Connected Layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out



## 3. Data Loading and Preparation

In this section, we will load the preprocessed data and prepare it for training. This involves defining the dataset structure, normalizing data, and splitting it into training and test sets.

### 3.1. Setting file path and loading data into a dataframe


In [ ]:
file_path = './data/raw_data/data_stock.csv'
data = pd.read_csv(file_path)

### 3.2. Giving ticker symbol for the training model

Specify in `input_value` the ticker value for which the model will be trained (e.g., 'MSFT' for Microsoft). 
Then, define a function to find the given index of the stock ticker in the DataFrame's columns.



In [ ]:
# Choose the stock to train by giving the ticker you want in input_value
input_value = 'MSFT'

def get_index(data, input_value):
    for index, value in enumerate(data):
        if input_value == value:
            return index
    return -1

index = get_index(data.columns, input_value)

### 3.3. Select the data to train on

We use 80% of the data for training and 20% for evaluating the model, that we convert in a numpy array for processing.


In [ ]:
training_row_index = int((len(data.iloc[:,index]))*0.8)
current_stock_data = data.iloc[:training_row_index, index]
current_stock_data = current_stock_data.to_numpy()


## 4. Model Training

Here we cover the process of training the GRU model. This includes defining the loss function, the optimizer, and the training loop where the model learns from the training data over several epochs.

### 4.1. Definition of a Dataset

For training the GRU model, we need to define a custom dataset. This involves creating a class that inherits from PyTorch's `Dataset` class. The dataset class will be responsible for loading the stock data, processing it, and returning samples in a format suitable for the GRU model.

In this section, we will define such a dataset class.



In [ ]:
class StockDataset(Dataset):
    """ Personnalised dataset for stock prices """
    def __init__(self, data, window_size):
        self.data = data
        self.window_size = window_size

    def __len__(self):
        return len(self.data) - self.window_size

    def __getitem__(self, idx):
        seq = self.data[idx:idx + self.window_size]
        label = self.data[idx + self.window_size]
        return torch.tensor(seq, dtype=torch.float).unsqueeze(-1), torch.tensor(label, dtype=torch.float)

The `StockDataset` class is defined to handle stock price data. It includes the following methods:

1. `__init__`: Constructor for the class, initializes the dataset with stock price data and a specified window size for creating sequences.
2. `__len__`: Returns the total number of samples in the dataset.
3. `__getitem__`: Retrieves a single sample from the dataset at a given index. This method returns a sequence of data points for the input and the next data point as the label.

You will need to further develop this class according to your specific requirements, such as data normalization, sequence creation, and handling the input and target data for the GRU model.

### 4.2. Instanciation of the model

#### 4.2.1 Define the window size
Here, we use the last 60 days of data to predict the stock price of the next day.  

Creating the dataset and DataLoader for training. The DataLoader fetches batches from the StockDataset without shuffling, because order matters in time series.

Instanciation of the GRU model with specific dimensions and layers.

Defining the loss function (Mean Squared Error Loss) and the optimizer (Adam) for the training.

Setting the number of epochs for training.

In [ ]:
window_size = 60

#### 4.2.2. Create the dataset & DataLoader for training
The DataLoader fetches batches from the StockDataset without shuffling, because order matters in time series.

In [ ]:
stock_dataset = StockDataset(current_stock_data, window_size)
train_loader = DataLoader(stock_dataset, batch_size=batch_size, shuffle=False)

#### 4.2.3. Instantiating the GRU model

Specific dimensions and layers

In [ ]:
model = GRUNet(input_dim=1, hidden_dim=100, output_dim=1, num_layers=2)

#### 4.2.4. Defining the loss function & the optimizer

In our case, we chose the Mean Square Error Loss & the Adam optimizer 

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### 4.2.5. Setting the number of epochs

Epochs designs the number of complete iteration through a dataset during the training of a model. In our case, we have determined that the optimal number of epochs was 93.

This is valid for the following ticker & hyperparameters:

In [ ]:
input_value = "AAPL"
learning_rate: 0.001
batch_size: 32
num_epochs: 93
hidden_dim: 100
num_layers: 2

#### 4.2.6. Training loop

The model is trained over multiple epochs. In each epoch, the model processes batches of data, calculates the loss, and updates its parameters.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

## 5. Saving the model

For the ticker put in `input_value`, we save the trained model state. The name of the `.pth` file allows to retrace which model is done for which action. For now, we have trained models for the `AAPL`, `AMZN` & `MSFT` stocks.

In [ ]:
torch.save(model.state_dict(), "models/model_"+ str(input_value) +".pth")